<center style="font-size: 2em; font-weight: bold;"> Determinantes del crimen </center>
<center style="font-size: 2em; font-weight: bold;"> Analisis Panel Data  </center>
<left style="font-size: 4em; font-weight: bold;"> Autor: Edinson Tolentino </left>

# Librerias

In [1]:
import pandas
import numpy
import os
import sys
import numpy

import statsmodels.api as sm
from statsmodels.iolib.summary2 import summary_col

import sklearn
import sklearn.compose
#import category_encoders
import sklearn.impute

from plotnine import *
from stargazer.stargazer import Stargazer
from IPython.core.display import HTML

#import py_hep_functions
import seaborn
import matplotlib.pyplot as plt

In [2]:

color = ["#3a5e8cFF", "#10a53dFF", "#541352FF", "#ffcf20FF", "#2f9aa0FF"]

def seq(start: float, stop: float, by: float, round_n=3) -> list:
    """
    Custom function, used for setting the breaks of plotnine scales.

       Parameters
    ----------
    start : float
        Start of the breaks.
    stop : float
        End of the breaks.
    by : float
        Steps between breaks.
    round_n: int, default=3
        Decimals to round floats in output.
    """
    epsilon = numpy.finfo("float").eps

    return [
        round(x, round_n) for x in list(numpy.arange(start, stop + (by - epsilon), by))
    ]
    
def normalize_frame(frame):
    """Normalize the data frame to make it performant and compatible with
    downstream libraries such as Scikit-Learn and CatBoost.
    
    In particular the following operations are performed:
    - Categorical levels that are not strings are converted into strings
    - Categorical missing values are converted into a distinct "N/A" level
    - Object-typed columns are converted into categorical columns
    """
    for col, series in frame.items():
        dtype = series.dtype
        if pandas.api.types.is_categorical_dtype(dtype):
            recode = False
            if series.hasnans:
                recode = True
            else:
                for value in series.cat.categories:
                    if not isinstance(value, str):
                        recode = True
                        break
            if recode:
                frame[col] = series.astype("str").fillna("N/A").astype(
                    "category")
        elif pandas.api.types.is_object_dtype(dtype):
            frame[col] = series.astype("str").fillna("N/A").astype("category")

    return frame

base_transformer = sklearn.pipeline.Pipeline([
    ("recode", sklearn.compose.ColumnTransformer(transformers=[
        ("category",
             #sklearn.preprocessing.OneHotEncoder(handle_unknown="ignore",
             sklearn.preprocessing.OneHotEncoder(drop="first",handle_unknown="ignore",                                                 
                min_frequency=0.01, max_categories=30, sparse_output= False),
             sklearn.compose.make_column_selector(dtype_exclude=numpy.number)
        ),
        ("number",
            sklearn.impute.SimpleImputer(strategy="mean"),
            sklearn.compose.make_column_selector(dtype_include=numpy.number)
        ),  
    ], n_jobs=-1)),
    #("rescale", sklearn.preprocessing.StandardScaler()),
])

In [3]:
output           = "C:/Users/et396/Dropbox/Docencia/Educate/Econometria/S4/Data"
os.chdir(output)

# Carga de data
base = pandas.read_stata("BD4.dta")

In [4]:
#from linearmodels.datasets import wage_panel
#
#data = wage_panel.load()
#data.info()

In [5]:
base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 504 entries, 0 to 503
Data columns (total 19 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   b       504 non-null    float32
 1   lb      504 non-null    float32
 2   cburg   504 non-null    float32
 3   sburg   504 non-null    float32
 4   ur      504 non-null    float32
 5   yr1     504 non-null    float32
 6   yr2     504 non-null    float32
 7   yr3     504 non-null    float32
 8   yr4     504 non-null    float32
 9   yr5     504 non-null    float32
 10  yr6     504 non-null    float32
 11  yr7     504 non-null    float32
 12  yr8     504 non-null    float32
 13  yr9     504 non-null    float32
 14  yr10    504 non-null    float32
 15  yr11    504 non-null    float32
 16  yr12    504 non-null    float32
 17  year    504 non-null    float32
 18  pfa     504 non-null    float32
dtypes: float32(19)
memory usage: 37.5 KB


In [6]:
base.describe().T

,count,mean,std,min,25%,50%,75%,max
b,504.0,14.978506,7.344829,4.121226,9.897938,12.768569,18.523424,46.137363
lb,504.0,2.601329,0.451574,1.416151,2.292326,2.546987,2.919036,3.831623
cburg,504.0,31.244047,8.584625,8.000000,26.000000,31.500000,37.000000,61.000000
sburg,504.0,10.547816,2.262820,5.200000,9.000000,10.300000,12.225000,17.400000
ur,504.0,11.316667,4.786645,1.800000,7.775000,10.750000,14.100000,26.100000
yr1,504.0,0.083333,0.276660,0.000000,0.000000,0.000000,0.000000,1.000000
yr2,504.0,0.083333,0.276660,0.000000,0.000000,0.000000,0.000000,1.000000
yr3,504.0,0.083333,0.276660,0.000000,0.000000,0.000000,0.000000,1.000000
yr4,504.0,0.083333,0.276660,0.000000,0.000000,0.000000,0.000000,1.000000
yr5,504.0,0.083333,0.276660,0.000000,0.000000,0.000000,0.000000,1.000000


In [7]:
# Lagged 
base['Lcburg']        = base.groupby('pfa')['cburg'].shift(-1)
base['Lsburg']        = base.groupby('pfa')['sburg'].shift(-1)
base['Llb']           = base.groupby('pfa')['lb'].shift(-1)

base['cpfa']  = base['pfa'].astype('object')
base['pfa']   = base['pfa'].astype('int64')
base['year']  = base['year'].astype('int64')


In [8]:
#base_final = base.dropna()
base_final = base.copy()
base_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 504 entries, 0 to 503
Data columns (total 23 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   b       504 non-null    float32
 1   lb      504 non-null    float32
 2   cburg   504 non-null    float32
 3   sburg   504 non-null    float32
 4   ur      504 non-null    float32
 5   yr1     504 non-null    float32
 6   yr2     504 non-null    float32
 7   yr3     504 non-null    float32
 8   yr4     504 non-null    float32
 9   yr5     504 non-null    float32
 10  yr6     504 non-null    float32
 11  yr7     504 non-null    float32
 12  yr8     504 non-null    float32
 13  yr9     504 non-null    float32
 14  yr10    504 non-null    float32
 15  yr11    504 non-null    float32
 16  yr12    504 non-null    float32
 17  year    504 non-null    int64  
 18  pfa     504 non-null    int64  
 19  Lcburg  462 non-null    float32
 20  Lsburg  462 non-null    float32
 21  Llb     462 non-null    float32
 22  cp

In [9]:
#print(base_final.descr)

In [10]:
data_panel = base_final.set_index(['pfa','year'])

# Filtro de variables
response = ['lb']
inputs = ['cburg', 'sburg','ur', 'Lcburg','Lsburg','Llb','yr1', 'yr2', 'yr3', 'yr4', 'yr5', 'yr6', 'yr7','yr8', 'yr9', 'yr10', 'yr11', 'yr12']
data_analysis = data_panel[response + inputs]

# Normalise informacion : number, category columns
data_panel = normalize_frame(data_analysis)
data_panel.info()


<class 'pandas.core.frame.DataFrame'>
MultiIndex: 504 entries, (1, 80) to (42, 91)
Data columns (total 19 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   lb      504 non-null    float32
 1   cburg   504 non-null    float32
 2   sburg   504 non-null    float32
 3   ur      504 non-null    float32
 4   Lcburg  462 non-null    float32
 5   Lsburg  462 non-null    float32
 6   Llb     462 non-null    float32
 7   yr1     504 non-null    float32
 8   yr2     504 non-null    float32
 9   yr3     504 non-null    float32
 10  yr4     504 non-null    float32
 11  yr5     504 non-null    float32
 12  yr6     504 non-null    float32
 13  yr7     504 non-null    float32
 14  yr8     504 non-null    float32
 15  yr9     504 non-null    float32
 16  yr10    504 non-null    float32
 17  yr11    504 non-null    float32
 18  yr12    504 non-null    float32
dtypes: float32(19)
memory usage: 40.2 KB


C:\Users\et396\AppData\Local\Temp\ipykernel_9228\2308428612.py:35: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead


In [11]:
# Filtro de variables
response = ['lb']
inputs = ['cburg', 'sburg','ur', 'Lcburg','Lsburg','Llb','yr1', 'yr2', 'yr3', 'yr4', 'yr5', 'yr6', 'yr7','yr8', 'yr9', 'yr10', 'yr11', 'yr12','cpfa']
data_analysis = base_final[response + inputs]

# Normalise informacion : number, category columns
data_fit = normalize_frame(data_analysis)
data_fit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 504 entries, 0 to 503
Data columns (total 20 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   lb      504 non-null    float32 
 1   cburg   504 non-null    float32 
 2   sburg   504 non-null    float32 
 3   ur      504 non-null    float32 
 4   Lcburg  462 non-null    float32 
 5   Lsburg  462 non-null    float32 
 6   Llb     462 non-null    float32 
 7   yr1     504 non-null    float32 
 8   yr2     504 non-null    float32 
 9   yr3     504 non-null    float32 
 10  yr4     504 non-null    float32 
 11  yr5     504 non-null    float32 
 12  yr6     504 non-null    float32 
 13  yr7     504 non-null    float32 
 14  yr8     504 non-null    float32 
 15  yr9     504 non-null    float32 
 16  yr10    504 non-null    float32 
 17  yr11    504 non-null    float32 
 18  yr12    504 non-null    float32 
 19  cpfa    504 non-null    category
dtypes: category(1), float32(19)
memory usage: 39.4 KB


C:\Users\et396\AppData\Local\Temp\ipykernel_9228\2308428612.py:35: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
C:\Users\et396\AppData\Local\Temp\ipykernel_9228\2308428612.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


# Pregunta 1

In [12]:
data_fit.describe().T

,count,mean,std,min,25%,50%,75%,max
lb,504.0,2.601329,0.451574,1.416151,2.292326,2.546987,2.919036,3.831623
cburg,504.0,31.244047,8.584625,8.000000,26.000000,31.500000,37.000000,61.000000
sburg,504.0,10.547816,2.262820,5.200000,9.000000,10.300000,12.225000,17.400000
ur,504.0,11.316667,4.786645,1.800000,7.775000,10.750000,14.100000,26.100000
Lcburg,462.0,30.841991,8.494922,8.000000,25.000000,31.000000,37.000000,61.000000
Lsburg,462.0,10.709090,2.240139,5.200000,9.125000,10.700000,12.375000,17.400000
Llb,462.0,2.632077,0.442546,1.566663,2.330205,2.559546,2.945509,3.831623
yr1,504.0,0.083333,0.276660,0.000000,0.000000,0.000000,0.000000,1.000000
yr2,504.0,0.083333,0.276660,0.000000,0.000000,0.000000,0.000000,1.000000
yr3,504.0,0.083333,0.276660,0.000000,0.000000,0.000000,0.000000,1.000000


In [13]:
data_panel.describe().T

,count,mean,std,min,25%,50%,75%,max
lb,504.0,2.601329,0.451574,1.416151,2.292326,2.546987,2.919036,3.831623
cburg,504.0,31.244047,8.584625,8.000000,26.000000,31.500000,37.000000,61.000000
sburg,504.0,10.547816,2.262820,5.200000,9.000000,10.300000,12.225000,17.400000
ur,504.0,11.316667,4.786645,1.800000,7.775000,10.750000,14.100000,26.100000
Lcburg,462.0,30.841991,8.494922,8.000000,25.000000,31.000000,37.000000,61.000000
Lsburg,462.0,10.709090,2.240139,5.200000,9.125000,10.700000,12.375000,17.400000
Llb,462.0,2.632077,0.442546,1.566663,2.330205,2.559546,2.945509,3.831623
yr1,504.0,0.083333,0.276660,0.000000,0.000000,0.000000,0.000000,1.000000
yr2,504.0,0.083333,0.276660,0.000000,0.000000,0.000000,0.000000,1.000000
yr3,504.0,0.083333,0.276660,0.000000,0.000000,0.000000,0.000000,1.000000


## Modelo LSDV

In [14]:
# Independed variabkes
response = data_fit[['lb']]

# Depended variables
predictors_cols1 = ['Lcburg', 'Lsburg', 'ur','yr3', 'yr4', 'yr5', 'yr6', 'yr7','yr8', 'yr9', 'yr10', 'yr11', 'yr12','cpfa' ]


In [15]:
base1        = data_fit[predictors_cols1]
predictors  = data_fit[predictors_cols1]
transformer = sklearn.clone(base_transformer)
inputs      = transformer.fit_transform(predictors)
inputs      = pandas.DataFrame(inputs, index=base1.index,
                               columns=transformer.get_feature_names_out())
inputs      = sm.add_constant(inputs, has_constant="raise")

m1 = sm.OLS( response, inputs, missing='drop').fit(cov_type="HC1")
print(m1.summary())

                            OLS Regression Results                            
Dep. Variable:                     lb   R-squared:                       0.851
Model:                            OLS   Adj. R-squared:                  0.837
Method:                 Least Squares   F-statistic:                     178.1
Date:                Sat, 30 Mar 2024   Prob (F-statistic):          6.09e-257
Time:                        17:55:26   Log-Likelihood:                 165.52
No. Observations:                 504   AIC:                            -245.0
Df Residuals:                     461   BIC:                            -63.47
Df Model:                          42                                         
Covariance Type:                  HC1                                         
                                        coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
const 

In [16]:
stargazer = Stargazer([m1])
stargazer.custom_columns(["lb"], [1])
stargazer.covariate_order(["number__Lcburg", 'number__Lsburg', "number__ur"])
stargazer.rename_covariates({"Intercept": "Constant"})
stargazer.add_line("Efectos fijos pfa", ["Yes"])
stargazer.add_line("Efectos fijos Year", ["Yes"])
stargazer

## Modelo Pooled

In [17]:
# Independed variabkes
response = data_panel[['lb']]

# Depended variables
predictors_cols1 = ['Lcburg', 'Lsburg', 'ur']


In [19]:
import statsmodels.api as sm
from linearmodels.panel import PooledOLS

base1       = data_panel[predictors_cols1]
predictors  = data_panel[predictors_cols1]
transformer = sklearn.clone(base_transformer)
inputs      = transformer.fit_transform(predictors)
inputs      = pandas.DataFrame(inputs, index=base1.index,
                               columns=transformer.get_feature_names_out())
inputs      = sm.add_constant(inputs, has_constant="raise")

m2 = PooledOLS( data_panel.lb, inputs).fit(cov_type='robust')
print(m2)

                          PooledOLS Estimation Summary                          
Dep. Variable:                     lb   R-squared:                        0.2722
Estimator:                  PooledOLS   R-squared (Between):              0.4728
No. Observations:                 504   R-squared (Within):              -0.5029
Date:                Sat, Mar 30 2024   R-squared (Overall):              0.2722
Time:                        17:56:42   Log-likelihood                   -233.91
Cov. Estimator:                Robust                                           
                                        F-statistic:                      62.319
Entities:                          42   P-value                           0.0000
Avg Obs:                       12.000   Distribution:                   F(3,500)
Min Obs:                       12.000                                           
Max Obs:                       12.000   F-statistic (robust):             77.075
                            

## Modelo de Efectos Fijos

In [ ]:
from linearmodels.panel import PanelOLS

m3 = PanelOLS( data_panel.lb, inputs, entity_effects=True, time_effects=True).fit(cov_type='robust')
print(m3)
#data_panel


## Modelo de Efectos aletorios

In [ ]:
from linearmodels.panel import RandomEffects

m4 = RandomEffects(data_panel.lb, inputs).fit(cov_type='robust')
print(m4)

# Comparacion de modelos

In [ ]:
from linearmodels.panel import compare

print(compare({"Pooled": m2, "FE": m3, "RE": m4}))

# Pregunta 2

## Pregunta 2a

In [ ]:
stargazer = Stargazer([m1])
stargazer.custom_columns(["lb"], [1])
stargazer.covariate_order(["number__Lcburg", 'number__Lsburg', "number__ur"])
stargazer.rename_covariates({"Intercept": "Constant"})
stargazer.add_line("Efectos fijos pfa", ["Yes"])
stargazer.add_line("Efectos fijos Year", ["Yes"])
stargazer

## Pregunta 2b

In [ ]:
print(m3)

# Pregunta 3
Test de Hausman, se constrasta el analisis del modelo RE

In [ ]:
print(m4)

# Pregunta 4

## Forma 1

In [ ]:
# Independed variabkes
response = data_panel[['lb']]

# Depended variables
predictors_cols1 = ['Llb','Lcburg', 'Lsburg', 'ur']


In [ ]:
from linearmodels.panel import PanelOLS


base1        = data_panel[predictors_cols1]
predictors  = data_panel[predictors_cols1]
transformer = sklearn.clone(base_transformer)
inputs      = transformer.fit_transform(predictors)
inputs      = pandas.DataFrame(inputs, index=base1.index,
                               columns=transformer.get_feature_names_out())
inputs      = sm.add_constant(inputs, has_constant="raise")

# Esdtimacion
m5 = PanelOLS( data_panel.lb, inputs, entity_effects=True, time_effects=True).fit(cov_type='robust')
print(m5)


In [ ]:
# semielasticidades
# delta 1
delta_1 = (m5.params['number__Lcburg'])/ (1-m5.params['number__Llb'])
delta_1


In [ ]:
# semielasticidades
# delta 2
delta_2 = (m5.params['number__Lsburg'])/ (1-m5.params['number__Llb'])
delta_2


In [ ]:
# semielasticidades
# delta 3
delta_3 = (m5.params['number__ur'])/ (1-m5.params['number__Llb'])
delta_3

## Forma 2

In [ ]:
# Independed variabkes
response = data_fit[['lb']]

# Depended variables
predictors_cols1 = ['Llb','Lcburg', 'Lsburg', 'ur','yr3', 'yr4', 'yr5', 'yr6', 'yr7','yr8', 'yr9', 'yr10', 'yr11', 'yr12','cpfa' ]


In [ ]:
base1        = data_fit[predictors_cols1]
predictors  = data_fit[predictors_cols1]
transformer = sklearn.clone(base_transformer)
inputs      = transformer.fit_transform(predictors)
inputs      = pandas.DataFrame(inputs, index=base1.index,
                               columns=transformer.get_feature_names_out())
inputs      = sm.add_constant(inputs, has_constant="raise")

m6 = sm.OLS( response, inputs).fit(cov_type="HC1")
print(m6.summary())

In [ ]:
stargazer = Stargazer([m1, m6])
stargazer.custom_columns(["lb",'lb'], [1,1])
stargazer.covariate_order(['number__Llb',"number__Lcburg", 'number__Lsburg', "number__ur"])
stargazer.rename_covariates({"Intercept": "Constant"})
stargazer.add_line("Efectos fijos pfa", ["Yes", "Yes"])
stargazer.add_line("Efectos fijos Year", ["Yes","Yes"])
stargazer

In [ ]:
# semielasticidades
# delta 1
delta_1 = (m6.params['number__Lcburg'])/ (1-m6.params['number__Llb'])
delta_1


In [ ]:
# semielasticidades
# delta 2
delta_2 = (m6.params['number__Lsburg'])/ (1-m6.params['number__Llb'])
delta_2

In [ ]:
# semielasticidades
# delta 3
delta_3 = (m6.params['number__ur'])/ (1-m6.params['number__Llb'])
delta_3